# Hyperparameter Tuning using HyperDrive

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [30]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.core import Environment

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling, GridParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

import requests
import pandas as pd

## Dataset
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'amazon_review_prediction_hyperdrive'

experiment=Experiment(ws, experiment_name) 

In [3]:
found = False
dataset_name = "Amazon Reviews"
dataset_descriptions = "Set of Amazon products text reviews together with their rating value ranging from 1 to 5."

if dataset_name in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[dataset_name] 

if not found:
        dataset = TabularDatasetFactory.from_delimited_files(path="https://github.com/iolucas/AMLE_P3/raw/main/amazon_reviews.csv") 
        dataset = ds.register(workspace=ws, name=dataset_name, description=dataset_descriptions)

In [4]:
df = ds.to_pandas_dataframe()
print(df.shape)
df.head()

(28332, 2)


,review,rating
0,I order 3 of them and one of the item is bad q...,3
1,Bulk is always the less expensive way to go fo...,4
2,Well they are not Duracell but for the price i...,5
3,Seem to work as well as name brand batteries a...,5
4,These batteries are very long lasting the pric...,5


In [40]:
df[:10]

,review,rating
0,I order 3 of them and one of the item is bad q...,3
1,Bulk is always the less expensive way to go fo...,4
2,Well they are not Duracell but for the price i...,5
3,Seem to work as well as name brand batteries a...,5
4,These batteries are very long lasting the pric...,5
5,Bought a lot of batteries for Christmas and th...,5
6,ive not had any problame with these batteries ...,5
7,Well if you are looking for cheap non-recharge...,5
8,These do not hold the amount of high power jui...,3
9,AmazonBasics AA AAA batteries have done well b...,4


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
amlcompute_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_v2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

In [6]:
#compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

In [8]:
from azureml.core import Environment

sklearn_env = Environment.get(ws, "AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")

In [12]:
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

param_sampling = GridParameterSampling({
    "--use-stopwords": choice(0, 1),
    "--use-binary-count": choice(0, 1),
    "--ngram-range": choice(1,2,3,4)  
})

estimator = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      arguments=[
                          #"--use-stopwords",
                          #"--use-binary-count",
                          #"--ngram-range", 1
                      ],
                      environment=sklearn_env
                     )

hyperdrive_run_config = HyperDriveConfig(
    run_config=estimator, 
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4
)

In [13]:
hdr = experiment.submit(config=hyperdrive_run_config)

## Run Details

In the cell below, use the `RunDetails` widget to show the different experiments.

In [14]:
RunDetails(hdr).show()
hdr.wait_for_completion()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

{'runId': 'HD_b1c3d548-8e1a-438c-b3c2-d8f44972d3f9',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-15T14:08:56.375988Z',
 'endTimeUtc': '2021-08-15T14:17:02.358529Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '1651d8f4-ef91-425f-9ce5-f540a83d13f0',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.32.0',
  'score': '0.8348332451032292',
  'best_child_run_id': 'HD_b1c3d548-8e1a-438c-b3c2-d8f44972d3f9_11',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg154595.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_b1c3d548-8e1a-438c-b3c2-d8f44972d3f9/azureml-logs/hyperdrive.txt?sv=2019-07-07&s

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [15]:
best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)

print('\n Accuracy:', best_run_metrics['Accuracy'])

for i in range(0,len(parameter_values),2):
    print(parameter_values[i], parameter_values[i+1])

Best Run Id:  HD_b1c3d548-8e1a-438c-b3c2-d8f44972d3f9_11

 Accuracy: 0.8348332451032292
--use-stopwords 1
--use-binary-count 0
--ngram-range 4


In [16]:
best_run.download_file('model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In the cell below, register the model, create an inference config and deploy the model as a web service.

In [17]:
from azureml.core import Model

model = Model.register(workspace=ws,
                       model_name='model.pkl',                # Name of the registered model in your workspace.
                       model_path='./model.pkl',  # Local file to upload and register as a model.
                      )


Registering model nb_model.pkl


In [18]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory=".", entry_script="score.py", environment=sklearn_env)

Warning, custom base image or base dockerfile detected without a specified `inferencing_stack_version`. Please set environment.inferencing_stack_version='latest'


In [19]:
service_name = 'hyperdrive-model'

service = model.deploy(ws, service_name, [model], overwrite=True, inference_config=inference_config)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-15 14:17:36+00:00 Creating Container Registry if not exists..
2021-08-15 14:27:36+00:00 Registering the environment.
2021-08-15 14:27:37+00:00 Use the existing image.
2021-08-15 14:27:38+00:00 Generating deployment configuration.
2021-08-15 14:27:39+00:00 Submitting deployment to compute..
2021-08-15 14:27:45+00:00 Checking the status of deployment my-sklearn-service..
2021-08-15 14:29:59+00:00 Checking the status of inference endpoint my-sklearn-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In the cell below, send a request to the web service you deployed to test it.

In [36]:
reviews = [
    "i loved it", 
    "fine product",
    "I hate it",
    "awful experience"
]

predictions = requests.post(service.scoring_uri, json={
    "data": reviews
}).json()

pd.DataFrame(data={"reviews":reviews, "Predicted Rating":predictions})

,reviews,Predicted Rating
0,i loved it,5
1,fine product,5
2,I hate it,5
3,awful experience,5


In the cell below, print the logs of the web service and delete the service

In [21]:
print(service.get_logs())

2021-08-15T14:29:44,541572338+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-08-15T14:29:44,541595739+00:00 - iot-server/run 
2021-08-15T14:29:44,744826635+00:00 - rsyslog/run 
2021-08-15T14:29:45,041000627+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-08-15T14:29:47,143754022+00:00 - iot-server/finish 1 0
2021-08-15T14:29:47,242975292+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (11)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 39
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-08-15 14:29:51,550 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-08-15 14:29:51,640 | root | INFO | Starting up request id generator
2021-08-15 14:29:51,640 | root | INFO | Star

In [ ]:
service.delete()